## 142 SAT [Enhancement] Märk ut saknade taggar för framkomlighetsbedömning

### version 2

## Analyze the trail  

* Issue [142 SAT Enhancement Märk ut saknade taggar för framkomlighetsbedömning](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/142)
 * Example pictures from the trail on a map
    * [SAT Arholma](https://wikimap.toolforge.org/?cat=SAT_Arholma&subcats=true&subcatdepth=3&cluster=false) - [OSM relation 19012436](https://www.openstreetmap.org/relation/19012436)
    * [SAT Lidö](https://wikimap.toolforge.org/?cat=SAT_Lidö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19020231](https://www.openstreetmap.org/relation/19020231)
    * [SAT Furusund](https://wikimap.toolforge.org/?cat=SAT_Furusund&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19016187](https://www.openstreetmap.org/relation/19016187)
    * [SAT Yxlan](https://wikimap.toolforge.org/?cat=SAT_Yxlan&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19020310](https://www.openstreetmap.org/relation/19020310)
    * [SAT Finnhamn](https://wikimap.toolforge.org/?cat=SAT_Finnhamn&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19018272](https://www.openstreetmap.org/relation/19018272)
    * [SAT Ingmarsö](https://wikimap.toolforge.org/?cat=SAT_Ingmarsö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19080874](https://www.openstreetmap.org/relation/19080874)
    * [SAT Brottö](https://wikimap.toolforge.org/?cat=SAT_Brottö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19141225](https://www.openstreetmap.org/relation/19141225)
    * [SAT Svartsö](https://wikimap.toolforge.org/?cat=SAT_Svartsö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19014515](https://www.openstreetmap.org/relation/19014515)
    * [SAT Grinda](https://wikimap.toolforge.org/?cat=SAT_Grinda&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19079703](https://www.openstreetmap.org/relation/19079703)
    * [SAT Möja](https://wikimap.toolforge.org/?cat=SAT_Möja&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19023630](https://www.openstreetmap.org/relation/19023630)
    * [SAT Sandhamn](https://wikimap.toolforge.org/?cat=SAT_Sandhamn&subcats=true&subcatdepth=3&cluster=false) - [OSM relation 19081125](https://www.openstreetmap.org/relation/19081125)
    * [SAT Runmarö](https://wikimap.toolforge.org/?cat=SAT_Runmar%C3%B6&subcats=true&subcatdepth=3&cluster=false) - OSM relation 19013472
    * [SAT Nämdö](https://wikimap.toolforge.org/?cat=SAT_N%C3%A4md%C3%B6&subcats=true&subcatdepth=3&cluster=false) - [OSM relation/19013473](https://www.openstreetmap.org/relation/19013473)
    * [SAT Ornö](https://wikimap.toolforge.org/?cat=SAT_Ornö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19023687](https://www.openstreetmap.org/relation/19023687)
    * [SAT Fjärdlång](https://wikimap.toolforge.org/?cat=SAT_Fjärdlång&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19016280](https://www.openstreetmap.org/relation/19016280)
    * [SAT Utö](https://wikimap.toolforge.org/?cat=SAT_Utö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19012654](https://www.openstreetmap.org/relation/19012654)
    * [SAT Ålö](https://wikimap.toolforge.org/?cat=SAT_Ålö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19012684](https://www.openstreetmap.org/relation/19012684)
    * [SAT Rånö](https://wikimap.toolforge.org/?cat=SAT_Rånö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19015969](https://www.openstreetmap.org/relation/19015969)
    * [SAT Nåttarö](https://wikimap.toolforge.org/?cat=SAT_Nåttarö&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19014571](https://www.openstreetmap.org/relation/19014571)
    * [SAT Landsort](https://wikimap.toolforge.org/?cat=SAT_Landsort&subcats=true&subcatdepth=3&cluster=false)- [OSM relation 19013576](https://www.openstreetmap.org/relation/19013576)

* Försök höja nivån på leddata exempel progress


In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))



Start: 2025-08-30 10:36:43


In [2]:
# ==== AUDIT: helpers + Commons integration (copy/paste this whole block) ====
import re
import html
import json
import requests
import pandas as pd
import geopandas as gpd
import folium
from functools import lru_cache
from urllib.parse import quote
from folium import GeoJson, GeoJsonTooltip, GeoJsonPopup

# ---------------------------- Config / Colors -------------------------------
AUDIT_COLORS = {
    "missing_surface": "#ff8800",  # orange
    "missing_access":  "#cc0000",  # red
    "missing_hike":    "#7e57c2",  # purple
    "inconsistent":    "#ffd54f",  # yellow
}

# ---------------------------- OSM utilities --------------------------------
def _mk_osm_links(osmid):
    try:
        osmid = int(osmid)
        view = f"https://www.openstreetmap.org/way/{osmid}"
        edit = f"https://www.openstreetmap.org/edit?editor=id&way={osmid}"
        return f'<a href="{view}" target="_blank">OSM way {osmid}</a> · <a href="{edit}" target="_blank">Öppna i iD</a>'
    except Exception:
        return ""

# ------------------------- Commons / MediaInfo utils ------------------------
def _is_http_url(s: str) -> bool:
    return isinstance(s, str) and s.lower().startswith(("http://", "https://"))

def _clean_str(s: str | None) -> str:
    return (s or "").strip()

def normalize_commons_title(image_tag: str) -> str | None:
    """
    Accepts OSM image=* values like:
      - File:Name.jpg
      - name.jpg
      - https://commons.wikimedia.org/wiki/File:Name.jpg
      - https://upload.wikimedia.org/.../Name.jpg (recovers best-effort)
    Returns canonical Commons page title like 'File:Name.jpg', or None if not Commons.
    """
    if not isinstance(image_tag, str) or not image_tag.strip():
        return None
    s = image_tag.strip()

    # If it's a Commons "File:" page or /wiki/File:
    m = re.search(r'(?:^|/wiki/)(File:[^#?]+)$', s, flags=re.I)
    if m:
        return "File:" + m.group(1).split("File:", 1)[1]

    # If it's a direct upload URL, try to extract the file name
    if "upload.wikimedia.org" in s:
        parts = s.split("/")
        if "thumb" in parts:
            # thumb URL ends with ".../WIDTHpx-Name.ext"
            last = parts[-1]
            last = re.sub(r'^\d+px-', '', last)  # remove leading dimension prefix
            return f"File:{last}"
        else:
            return f"File:{parts[-1]}"

    # Plain filename -> assume Commons file
    if not _is_http_url(s):
        return s if s.lower().startswith("file:") else f"File:{s}"

    # Non-Commons external URL
    return None

def commons_thumb_url(title: str, width: int = 600) -> str:
    """Robust thumbnail via Special:FilePath (server-side resize)."""
    safe_title = quote(title, safe="/:")
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{safe_title}?width={width}"

def commons_filepage_url(title: str) -> str:
    safe_title = quote(title.replace(" ", "_"), safe="/:")
    return f"https://commons.wikimedia.org/wiki/{safe_title}"

@lru_cache(maxsize=1024)
def get_commons_entity_json_for_title(title: str) -> dict | None:
    """
    Fetch MediaInfo JSON via Special:EntityData for the given file title.
    Returns parsed JSON or None on failure. Cached for performance.
    """
    if not _clean_str(title):
        return None
    url = f"https://commons.wikimedia.org/wiki/Special:EntityData/{quote(title, safe='/:')}.json"
    try:
        r = requests.get(url, timeout=15)
        if r.status_code == 200:
            return r.json()
    except Exception:
        pass
    return None

def commons_has_p10689(title: str, osmid: int | str) -> tuple[bool, str | None]:
    """
    Check if the Commons MediaInfo for 'title' contains P10689 = given OSM way id.
    Returns (True/False, mediainfo_id or None).
    """
    data = get_commons_entity_json_for_title(title)
    if not data or "entities" not in data:
        return (False, None)

    for mid, ent in data["entities"].items():
        stmts = ent.get("statements", {})
        p = stmts.get("P10689")
        if not p:
            continue
        for claim in p:
            snak = claim.get("mainsnak", {})
            dv = snak.get("datavalue", {})
            val = dv.get("value")
            if str(val) == str(osmid):
                return (True, mid)
    return (False, None)

def make_image_gallery_html(image_value: str | None, max_images: int = 3) -> str:
    """
    Create HTML for one or more images from OSM image=*.
    - Semicolon-separated values supported.
    - Commons links become clickable thumbnails (to file page).
    - Non-Commons http(s) URLs embedded directly.
    """
    if not _clean_str(image_value):
        return ""

    # Split by ';' or '|', keep order, dedupe while preserving order
    raw_items = re.split(r'[;|]', image_value)
    seen, items = set(), []
    for it in raw_items:
        s = _clean_str(it)
        if s and s not in seen:
            seen.add(s)
            items.append(s)
    items = items[:max_images]

    parts = []
    for s in items:
        title = normalize_commons_title(s)
        if title:
            thumb = commons_thumb_url(title, width=600)
            link = commons_filepage_url(title)
            parts.append(
                f'<a href="{html.escape(link)}" target="_blank">'
                f'<img src="{html.escape(thumb)}" style="max-width:100%;height:auto;border-radius:8px;margin:4px 0"/></a>'
            )
        elif _is_http_url(s):
            parts.append(
                f'<img src="{html.escape(s)}" style="max-width:100%;height:auto;border-radius:8px;margin:4px 0"/>'
            )
        # else: skip non-URL, non-Commons, non-filename
    return "".join(parts)

# --------------------------- Main audit builder -----------------------------
def build_audit_layers(seg_gdf_4326: gpd.GeoDataFrame, label: str, mode: str = "foot"):
    """
    Tar ett per-etapp seg_gdf (EPSG:4326) med kolumner:
    osmid, highway, surface, foot, bicycle, sac_scale, trail_visibility, tracktype, image, geometry
    Returnerar (layers:list[folium.FeatureGroup], audit_df:DataFrame)
    """
    # Säkerställ kolumner
    for col in ["osmid","highway","surface","foot","bicycle","sac_scale","trail_visibility","smoothness","tracktype","image"]:
        if col not in seg_gdf_4326.columns:
            seg_gdf_4326[col] = None

    # Normalisera text (stringify and lowercase)
    sg = seg_gdf_4326.copy()
    for col in ["highway","surface","foot","bicycle","sac_scale","trail_visibility","tracktype"]:
        sg[col] = sg[col].astype(str).str.lower().replace({"none": ""})

    # --- filter för luckor
    missing_surface = sg.loc[(sg["surface"] == "") | (sg["surface"].isna())]

    if mode == "foot":
        miss_access = sg.loc[
            (sg["highway"].isin(["path","footway"])) &
            (sg["foot"].isin(["", "nan"]) | sg["foot"].isna())
        ]
    else:
        miss_access = sg.loc[
            (sg["bicycle"].isin(["","nan"]) | sg["bicycle"].isna())
        ]

    miss_hike = sg.loc[
        sg["highway"].isin(["path","footway"]) &
        (
            (sg["sac_scale"].isin(["","nan"]) | sg["sac_scale"].isna()) |
            (sg["trail_visibility"].isin(["","nan"]) | sg["trail_visibility"].isna())
        )
    ]

    inconsistent = sg.loc[
        (sg["tracktype"].isin(["grade1","grade2","grade3","grade4","grade5"])) &
        (sg["highway"] != "track")
    ]

    # --- audit dataframe (en rad per way)
    base = sg[["osmid","highway","surface","tracktype","foot","bicycle","sac_scale","trail_visibility","smoothness","image","geometry"]].copy()
    base["missing_surface"] = base.index.isin(missing_surface.index)
    base["missing_access"]  = base.index.isin(miss_access.index)
    base["missing_hike"]    = base.index.isin(miss_hike.index)
    base["inconsistent"]    = base.index.isin(inconsistent.index)
    base["Etapp"]           = label

    audit_df = pd.DataFrame(base.drop(columns=["geometry"]))
    audit_df["osmid"] = pd.to_numeric(audit_df["osmid"], errors="coerce").astype("Int64")

    # --- Enrichment for popup: OSM links, image gallery, Commons P10689 check
    def _popup_enrichment(row):
        osm_link = _mk_osm_links(row.get("osmid"))
        image_html = make_image_gallery_html(row.get("image"))

        commons_status = ""
        # For P10689 check, use the FIRST Commons-like title we can normalize
        first_title = None
        imgs = re.split(r'[;|]', _clean_str(row.get("image")))
        for im in imgs:
            t = normalize_commons_title(_clean_str(im))
            if t:
                first_title = t
                break

        if first_title and row.get("osmid"):
            try:
                ok, mid = commons_has_p10689(first_title, row.get("osmid"))
                if ok:
                    commons_status = f'✅ Commons (P10689) länkar till OSM way {row.get("osmid")} (MediaInfo {html.escape(str(mid))})'
                else:
                    commons_status = f'⚠️ Commons saknar P10689={row.get("osmid")} på bilden ({html.escape(first_title)})'
            except Exception as e:
                commons_status = f'⚠️ Kunde inte kontrollera Commons P10689 ({html.escape(str(e))})'

        return pd.Series({
            "osm_link": osm_link,
            "image_html": image_html,
            "commons_p10689": commons_status
        })

    sg_popup = sg.copy()
    enrich = sg_popup.apply(_popup_enrichment, axis=1)
    for c in ["osm_link","image_html","commons_p10689"]:
        sg_popup[c] = enrich[c]

    # --- bygg Folium-lager
    layers = []
    def _make_layer(df, name, color, _tooltip_text):
        if df.empty:
            return None
        df2 = sg_popup.loc[df.index].copy()
        gj = GeoJson(
            df2[[
                "geometry","osm_link","image_html","commons_p10689",
                "highway","surface","tracktype","foot","bicycle"
            ]],
            name=name,
            style_function=lambda _f: {"weight": 6, "color": color},
            tooltip=GeoJsonTooltip(
                fields=["highway","surface","tracktype"],
                aliases=["highway","surface","tracktype"]
            ),
            popup=GeoJsonPopup(
                fields=["osm_link","image_html","commons_p10689","highway","surface","tracktype","foot","bicycle"],
                aliases=["", "", "", "highway","surface","tracktype","foot","bicycle"],
                parse_html=True
            )
        )
        fg = folium.FeatureGroup(name=name)
        gj.add_to(fg)
        return fg

    L1 = _make_layer(missing_surface, f"Saknar surface: {label}", AUDIT_COLORS["missing_surface"], "Saknar surface=*")
    L2 = _make_layer(miss_access,  f"Saknar {'foot' if mode=='foot' else 'bicycle'}: {label}", AUDIT_COLORS["missing_access"], "Saknar access-tagg")
    L3 = _make_layer(miss_hike,    f"Saknar sac/visibility: {label}", AUDIT_COLORS["missing_hike"], "Saknar vandringstaggar")
    L4 = _make_layer(inconsistent, f"Inkonsekvent tracktype: {label}", AUDIT_COLORS["inconsistent"], "tracktype utan highway=track")

    for L in [L1, L2, L3, L4]:
        if L is not None:
            layers.append(L)

    return layers, audit_df

# ------------------------------ Legend widget -------------------------------
def add_audit_legend(m: folium.Map):
    html_legend = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index: 9999;
                background: rgba(255,255,255,0.92); padding: 10px 12px; border-radius: 10px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.15); font-size: 13px;">
      <div style="font-weight:600; margin-bottom:6px;">Datakvalitet (saknade taggar)</div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS['missing_surface']};margin-right:6px;"></span>saknar <code>surface=*</code></div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS['missing_access']};margin-right:6px;"></span>saknar <code>foot=*</code> / <code>bicycle=*</code></div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS['missing_hike']};margin-right:6px;"></span>saknar <code>sac_scale</code> / <code>trail_visibility</code></div>
      <div><span style="display:inline-block;width:14px;height:3px;background:{AUDIT_COLORS['inconsistent']};margin-right:6px;"></span><code>tracktype=*</code> utan <code>highway=track</code></div>
    </div>
    """
    # Keep map happy with a no-op and add legend
    folium.Marker(location=[0, 0], opacity=0)
    m.get_root().html.add_child(folium.Element(html_legend))


In [3]:

# ======================= SAT: Hämta etapper (WD) + geometri (OSM) =======================
import os
import json
import time
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

WD_ENDPOINT = "https://query.wikidata.org/sparql"
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# ---- SPARQL: hämta etapper för SAT ------------------------------------------------------
def fetch_sat_sections_from_wikidata(trail_qid: str = "Q131318799") -> pd.DataFrame:
    """
    Returnerar DataFrame med kolumner:
    ['section_qid','sectionLabel','osmr','osmw'] där osmr = OSM relation-id (P402, Int64), osmw = OSM way-id (P10689/P11693, Int64)
    """
    query = f"""
    SELECT ?section ?sectionLabel ?osmr ?osmw_p10689 ?osmw_p11693 WHERE {{
      VALUES ?trail {{ wd:{trail_qid} }}
      FILTER EXISTS {{ ?trail ?p ?o }}  # hint: force trail to be read

      # Hämta etapper: antingen "part of" (P361+) eller "has part" (P527)
      {{
        ?section wdt:P361+ ?trail .
      }} UNION {{
        ?trail wdt:P527 ?section .
      }}
      FILTER(?section != ?trail)

      OPTIONAL {{ ?section wdt:P402 ?osmr }}             # OSM relation id
      OPTIONAL {{ ?section wdt:P10689 ?osmw_p10689 }}    # OSM way id (alt 1)
      OPTIONAL {{ ?section wdt:P11693 ?osmw_p11693 }}    # OSM way id (alt 2 – vissa objekt använder denna)

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    ORDER BY ?sectionLabel
    """
    headers = {"Accept": "application/sparql-results+json", "User-Agent": "SAT-etapper/1.0 (tools)"}
    r = requests.get(WD_ENDPOINT, params={"query": query}, headers=headers, timeout=60)
    r.raise_for_status()
    data = r.json()

    rows = []
    for b in data["results"]["bindings"]:
        qid = b["section"]["value"].split("/")[-1]
        label = b.get("sectionLabel", {}).get("value", qid)
        osmr = b.get("osmr", {}).get("value")
        osmw1 = b.get("osmw_p10689", {}).get("value")
        osmw2 = b.get("osmw_p11693", {}).get("value")
        # välj way-id om något finns
        osmw = osmw1 or osmw2
        rows.append({
            "section_qid": qid,
            "sectionLabel": label,
            "osmr": pd.to_numeric(osmr, errors="coerce"),
            "osmw": pd.to_numeric(osmw, errors="coerce"),
        })
    df = pd.DataFrame(rows)
    if not df.empty:
        df["osmr"] = df["osmr"].astype("Int64")
        df["osmw"] = df["osmw"].astype("Int64")
    return df

# ---- Overpass helpers ---------------------------------------------------------------
def _overpass(query: str, retries: int = 3, pause: float = 2.0) -> dict:
    headers = {"User-Agent": "SAT-etapper/1.0 (tools)"}
    for i in range(retries):
        try:
            resp = requests.post(OVERPASS_URL, data={"data": query}, headers=headers, timeout=120)
            if resp.status_code == 429 and i < retries-1:
                time.sleep(pause * (i+1))
                continue
            resp.raise_for_status()
            return resp.json()
        except Exception as e:
            if i == retries-1:
                raise
            time.sleep(pause * (i+1))
    return {"elements": []}

def fetch_ways_for_relation(rel_id: int) -> list[dict]:
    """
    Hämtar alla member ways för en given relation och returnerar Overpass 'elements' för ways.
    """
    q = f"""
    [out:json][timeout:120];
    relation({rel_id});
    way(r);
    out tags geom;
    """
    data = _overpass(q)
    return [el for el in data.get("elements", []) if el.get("type") == "way"]

def fetch_way(way_id: int) -> list[dict]:
    """
    Hämtar en enskild way (som lista med ett element för enhetlighet).
    """
    q = f"""
    [out:json][timeout:120];
    way({way_id});
    out tags geom;
    """
    data = _overpass(q)
    return [el for el in data.get("elements", []) if el.get("type") == "way"]

# ---- Konvertera Overpass-way -> GeoDataFrame -----------------------------------------
_CORE_TAGS = [
    "name","highway","surface","tracktype","foot","bicycle",
    "sac_scale","trail_visibility","smoothness","width",
    "mtb:scale","mtb:scale:imba","osmc:symbol","image"
]

def _way_to_feature(el: dict) -> dict | None:
    """
    Gör om Overpass way-element till dict med 'osmid', 'tags' och 'geometry' (LineString).
    """
    if "geometry" not in el:
        return None
    coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
    if len(coords) < 2:
        return None
    tags = el.get("tags", {}) or {}
    props = {"osmid": el["id"]}
    for k in _CORE_TAGS:
        props[k] = tags.get(k)
    return {"properties": props, "geometry": LineString(coords)}

def elements_to_gdf(elements: list[dict]) -> gpd.GeoDataFrame:
    feats = []
    for el in elements:
        f = _way_to_feature(el)
        if f:
            feats.append(f)
    if not feats:
        return gpd.GeoDataFrame(columns=["osmid",*_CORE_TAGS,"geometry"], geometry="geometry", crs="EPSG:4326")
    df = pd.DataFrame([f["properties"] for f in feats])
    gdf = gpd.GeoDataFrame(df, geometry=[f["geometry"] for f in feats], crs="EPSG:4326")
    # Säkerställ alla kolumner finns
    for k in ["osmid",*_CORE_TAGS]:
        if k not in gdf.columns:
            gdf[k] = None
    # Namn-fallback: använd relation/etapp-namn senare om 'name' saknas
    return gdf

# ---- Huvud: bygg seg_gdf_4326 per etapp --------------------------------------------
def build_sat_seg_gdfs(trail_qid: str = "Q131318799", prefer_relation: bool = True) -> dict[str, gpd.GeoDataFrame]:
    """
    Returnerar dict: { 'Etappnamn': seg_gdf_4326 } där varje GeoDataFrame är EPSG:4326 och
    innehåller kolumnerna:
    ['osmid','name','highway','surface','tracktype','foot','bicycle','sac_scale',
     'trail_visibility','smoothness','width','mtb:scale','mtb:scale:imba','osmc:symbol','image','geometry']
    """
    meta = fetch_sat_sections_from_wikidata(trail_qid)
    if meta.empty:
        print("Hittade inga etapper i Wikidata för", trail_qid)
        return {}

    out: dict[str, gpd.GeoDataFrame] = {}
    for _, row in meta.iterrows():
        label = str(row["sectionLabel"]).strip() or row["section_qid"]
        rel_id = row["osmr"] if pd.notna(row["osmr"]) else None
        way_id = row["osmw"] if pd.notna(row["osmw"]) else None

        elements: list[dict] = []
        if prefer_relation and rel_id:
            try:
                elements = fetch_ways_for_relation(int(rel_id))
            except Exception as e:
                print(f"[{label}] Fel vid hämtning av relation {rel_id}: {e}")

        # fallback till way-id om fångade noll/inget via relation
        if (not elements) and way_id:
            try:
                elements = fetch_way(int(way_id))
            except Exception as e:
                print(f"[{label}] Fel vid hämtning av way {way_id}: {e}")

        gdf = elements_to_gdf(elements)

        # Fyll 'name' om saknas med etappens label
        if "name" not in gdf.columns:
            gdf["name"] = None
        gdf["name"] = gdf["name"].fillna(label)

        out[label] = gdf

    return out

# ======================= EXEMPEL: bygg + spara kartor per etapp =========================
# Antag att du redan har make_audit_map(seg_gdf_4326, label, mode, OUTPUT_DIR, PROJECT_NAME, stamp)
# från tidigare cell. Nedan kör hämtning + generering av en karta per etapp.

# Parametrar för sparning
PROJECT_NAME = "SAT_142_2_per_etapp"
OUTPUT_DIR = "../kartor"
stamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M")

# 1) Hämta seg_gdf_4326 per etapp
etapp_gdfs = build_sat_seg_gdfs(trail_qid="Q131318799", prefer_relation=True)



In [4]:
# ===================== EN KARTA FÖR ALLA ETAPPER ============================
# Förutsätter att du redan har:
# - etapp_gdfs: dict[str, GeoDataFrame], där varje gdf är EPSG:4326 och har kolumnerna som build_audit_layers förväntar sig
# - build_audit_layers(...) och add_audit_legend(...) från tidigare block
# - OUTPUT_DIR, PROJECT_NAME, stamp definierade

import os
import folium
import geopandas as gpd
import pandas as pd

def make_audit_map_all_etapper(
    etapp_gdfs: dict,
    mode: str = "foot",
    OUTPUT_DIR: str = "..",
    PROJECT_NAME: str = "SAT",
    stamp: str = ""
):
    # 1) Skapa Folium-kartan
    m = folium.Map(location=[59.33, 18.06], zoom_start=8, control_scale=True, tiles=None)
    folium.TileLayer("OpenStreetMap", name="OSM").add_to(m)
    folium.TileLayer("CartoDB Positron", name="Ljus").add_to(m)
    folium.TileLayer("CartoDB Dark_Matter", name="Mörk").add_to(m)

    # 2) Lägg till lager för varje etapp och samla audit-data
    audit_frames = []
    any_geometry = []

    for etapp_label, seg_gdf_4326 in etapp_gdfs.items():
        if seg_gdf_4326 is None or seg_gdf_4326.empty:
            continue

        # Bygg revisionslager (med bild i popup och Commons P10689-kontroll)
        layers, audit_df = build_audit_layers(seg_gdf_4326=seg_gdf_4326, label=etapp_label, mode=mode)
        audit_frames.append(audit_df)

        # Lägg till alla lagren i kartan (namnen inkluderar redan etappens label)
        for L in layers:
            L.add_to(m)

        # Spara geometrier för fit_bounds
        try:
            any_geometry.append(seg_gdf_4326[seg_gdf_4326.geometry.notnull()].geometry)
        except Exception:
            pass

        # (Valfritt) tunn grå översiktslinje per etapp för kontext
        try:
            gj_outline = folium.GeoJson(
                data=seg_gdf_4326[["geometry"]],
                name=f"Översikt: {etapp_label}",
                style_function=lambda _f: {"weight": 2, "color": "#777", "opacity": 0.6}
            )
            gj_outline.add_to(m)
        except Exception:
            pass

    # 3) Fit bounds över alla etapper
    try:
        if any_geometry:
            all_geom = gpd.GeoSeries(pd.concat(any_geometry, ignore_index=True), crs="EPSG:4326")
            if not all_geom.empty and all_geom.is_valid.any():
                bounds = all_geom.total_bounds  # [minx, miny, maxx, maxy]
                m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
    except Exception:
        pass

    # 4) Legend och lagerkontroll
    add_audit_legend(m)
    folium.LayerControl(collapsed=False).add_to(m)

    # 5) Spara karta + sammanställd audit-tabell
    print(OUTPUT_DIR)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    html_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_ALL_{mode}_{stamp}.html")
    m.save(html_path)

    audit_all = pd.concat(audit_frames, ignore_index=True) if audit_frames else pd.DataFrame()
    csv_path = os.path.join(OUTPUT_DIR, f"{PROJECT_NAME}_ALL_{mode}_{stamp}.csv")
    try:
        audit_all.to_csv(csv_path, index=False)
    except Exception:
        pass

    return html_path, csv_path, audit_all

# ===================== ANROPA: EN KARTA FÖR ALLA ============================
# Skapar EN html-karta för alla etapper och en CSV med audit-data
all_html, all_csv, audit_df_all = make_audit_map_all_etapper(
    etapp_gdfs=etapp_gdfs,
    mode="foot",
    OUTPUT_DIR=OUTPUT_DIR,
    PROJECT_NAME=PROJECT_NAME,
    stamp=stamp
)

print("✅ Skapade gemensam karta:", all_html)
print("📄 Samlad audit-CSV:", all_csv)
try:
    display(audit_df_all.head())
except NameError:
    pass


../kartor
✅ Skapade gemensam karta: ../kartor/SAT_142_2_per_etapp_ALL_foot_20250830_1036.html
📄 Samlad audit-CSV: ../kartor/SAT_142_2_per_etapp_ALL_foot_20250830_1036.csv


,osmid,highway,surface,tracktype,foot,bicycle,sac_scale,trail_visibility,smoothness,image,missing_surface,missing_access,missing_hike,inconsistent,Etapp
0,39504808,,,,,,,,None,https://upload.wikimedia.org/wikipedia/commons...,True,False,False,False,Roddbåtar att korsa mellan Finnhamn till Ingmarsö
1,39504808,,,,,,,,None,https://upload.wikimedia.org/wikipedia/commons...,True,False,False,False,Roddbåtar att korsa mellan Ingmarsö till Finnhamn
2,42559813,service,,,,,,,None,None,True,False,False,False,SAT Arholma
3,42559822,tertiary,gravel,,,,,,intermediate,None,False,False,False,False,SAT Arholma
4,42701072,service,,,yes,yes,,,None,None,True,False,False,False,SAT Arholma


In [5]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))


Date: 2025-08-30 10:37:01
Total time elapsed: 17.93 seconds
